In [1]:
# Importar librerias
import wfdb.plot
import wfdb.processing
import numpy as np
import wfdb
from wfdb import processing, rdrecord, rdann

# Cargar datos

El problema que tenemos al trabajar con la base de datos MIT-BIH es que se encuentra en un formato basado en 4 archivos. Para facilitar el trabajo, se identificarán los nombres de los archivos de la base de datos MIT-BIH y se creará una función que permita cargar los datos de manera más sencilla.

In [2]:
import os

def listar_archivos(carpeta):
    try:
        archivos = os.listdir(carpeta)
        archivos_correctos = []
        
        for archivo in archivos:
            if archivo.endswith('.atr'):
                archivos_correctos.append(f"{carpeta}{archivo.replace('.atr', '')}")
        return archivos_correctos
    except FileNotFoundError:
        print("La carpeta no existe")

In [3]:
carpeta = 'data/mit-bih-arrhythmia-database-1.0.0/'
nombre_archivos = listar_archivos(carpeta)

Una vez tenemos los nombres de los archivos, procedemos a calcular el ritmo cardiaco de cada uno de los registros.

In [4]:
import pandas as pd
from hr_engine import get_features

results = []
def calcular_ritmo_cardiaco(file_name):
    sample_size = 650000
    record = wfdb.rdrecord(file_name, sampto=sample_size)
    annotation = wfdb.rdann(file_name, 'atr', sampto=sample_size)
    annotation_types = set(annotation.symbol)
    arrhythmic_symbols = {'V', 'A', 'L', 'R', 'f', 'j'}
    diagnosis = 1 if any(symbol in annotation_types for symbol in arrhythmic_symbols) else 0
    fs = record.fs
    r_peaks = annotation.sample
    rr_intervals = np.diff(r_peaks) / fs
    
    # IQR filtering for outliers in RR intervals
    q1, q3 = np.percentile(rr_intervals, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    rr_intervals_filtered = rr_intervals[(rr_intervals >= lower_bound) & (rr_intervals <= upper_bound)]
    
    # Calculate HR values after filtering
    hr_values = 60 / rr_intervals_filtered
    time = r_peaks[1:] / fs  # Keep original time for corresponding RR intervals
    time_filtered = time[(rr_intervals >= lower_bound) & (rr_intervals <= upper_bound)]
    
    # Ensure `hr_values` and `time_filtered` are aligned
    hr_df = pd.DataFrame({'time': time_filtered, 'hr': hr_values})
    hr_df = hr_df[(hr_df['hr'] > 30) & (hr_df['hr'] < 220)].reset_index(drop=True)
    
    # Apply median smoothing over a 5-point window on the filtered HR values
    hr_values_smoothed = hr_df['hr'].rolling(window=5, center=True).median().dropna().values
    time_smoothed = hr_df['time'][2:-2].values  # Adjust time to match smoothed HR length
    results.append(get_features(hr_values=hr_values_smoothed, time_hr=time_smoothed, record_name=file_name, diagnosis=diagnosis))
    

In [5]:
# heart_rate_values = [calcular_ritmo_cardiaco(file_name) for file_name in nombre_archivos ]
for file_name in nombre_archivos:
    calcular_ritmo_cardiaco(file_name)
df = pd.DataFrame(results)
df.head()

/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hr_smoothed = pd.Series(hr_values).rolling(window=window_size, center=True).mean().fillna(method='bfill').fillna(
/home/gino/Documents/Repos/pia-02/hr_engine.py:137: FutureWarning: Series.fillna with 'method' is

,patient,diagnosis,highest_heart_rate,lowest_heart_rate,mean_heart_rate,median_heart_rate,standard_deviation_hr,minimum_hrv,maximum_hrv,mean_hrv,...,hf_power,mean_rr,standard_deviation_rr,minimum_rr,maximum_rr,mean_deviation,tendency_standard_deviation,approximation_entropy,sample_entropy,outliers_percentage
0,data/mit-bih-arrhythmia-database-1.0.0/213,1,113.089005,102.857143,108.127408,108.542714,2.022095,0.501672,5.685571,0.645095,...,22135.071058,3.906391,2.105712,1.069444,12.505556,4.251918e-15,0.702512,1.408457,1.177573,0.062168
1,data/mit-bih-arrhythmia-database-1.0.0/209,1,113.684211,84.705882,96.300366,95.154185,5.026822,0.346988,12.139094,1.013533,...,36701.641292,6.039207,3.036317,1.075000,33.636111,1.167013e-14,4.329784,1.282741,1.121386,0.084470
2,data/mit-bih-arrhythmia-database-1.0.0/214,1,90.376569,64.094955,75.858277,75.789474,4.681641,0.198950,12.804878,1.090299,...,31168.361433,10.013405,6.048494,2.150000,30.163889,4.313870e-15,3.849098,1.307559,1.043753,0.092603
3,data/mit-bih-arrhythmia-database-1.0.0/100,1,84.375000,70.819672,75.502196,75.000000,2.644870,0.237619,5.898876,0.629692,...,20908.503694,6.649409,2.325983,1.527778,18.361111,-7.781047e-15,2.302985,1.226100,1.048676,0.088654
4,data/mit-bih-arrhythmia-database-1.0.0/117,1,54.000000,47.893570,50.786761,50.823529,0.903975,0.109323,2.010028,0.251715,...,6732.386314,10.148943,5.530223,2.288889,30.327778,-3.934648e-15,0.644782,1.558320,1.377851,0.093306


Visualizacion

In [6]:
# Save the results to a CSV file
df.to_csv('mit-bih-data-clear-data.csv', index=False)

In [7]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(36, 18))
# for i, dataframe in enumerate(heart_rate_values):
#     plt.plot(dataframe['time'], dataframe['hr'], label=f'Archivo {i + 1}')
# 
# plt.xlabel("Tiempo (segundos)")
# plt.ylabel("Ritmo Cardíaco (bpm)")
# plt.title("Comparación del Ritmo Cardíaco a lo Largo del Tiempo")
# plt.legend()
# plt.grid()
# plt.show()